https://www.kaggle.com/c/pubg-finish-placement-prediction

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras

In [ ]:
pd.options.display.max_columns = 30

# Import Data And EDA

In [ ]:
train_data = pd.read_csv('../input/train.csv')

In [ ]:
train_data.head()

In [ ]:
train_data.shape

In [ ]:
train_data.describe()

### Data fields
DBNOs - Number of enemy players knocked.

assists - Number of enemy players this player damaged that were killed by teammates.

boosts - Number of boost items used.

damageDealt - Total damage dealt. Note: Self inflicted damage is subtracted.

headshotKills - Number of enemy players killed with headshots.

heals - Number of healing items used.

killPlace - Ranking in match of number of enemy players killed.

killPoints - Kills-based external ranking of player. (Think of this as an Elo ranking where only kills matter.)

killStreaks - Max number of enemy players killed in a short amount of time.

kills - Number of enemy players killed.

longestKill - Longest distance between player and player killed at time of death. This may be misleading, as downing a player and driving away may lead to a large longestKill stat.

matchId - Integer ID to identify match. There are no matches that are in both the training and testing set.

revives - Number of times this player revived teammates.

rideDistance - Total distance traveled in vehicles measured in meters.

roadKills - Number of kills while in a vehicle.

swimDistance - Total distance traveled by swimming measured in meters.

teamKills - Number of times this player killed a teammate.

vehicleDestroys - Number of vehicles destroyed.

walkDistance - Total distance traveled on foot measured in meters.

weaponsAcquired - Number of weapons picked up.

winPoints - Win-based external ranking of player. (Think of this as an Elo ranking where only winning matters.)

groupId - Integer ID to identify a group within a match. If the same group of players plays in different matches, they will have a different groupId each time.

numGroups - Number of groups we have data for in the match.

maxPlace - Worst placement we have data for in the match. This may not match with numGroups, as sometimes the data skips over placements.

winPlacePerc - The target of prediction. This is a percentile winning placement, where 1 corresponds to 1st place, and 0 corresponds to last place in the match. It is calculated off of maxPlace, not numGroups, so it is possible to have missing chunks in a match.

In [ ]:
train_data.columns

In [ ]:
X = train_data[['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills',
                'killStreaks', 'longestKill', 'maxPlace', 'numGroups', 'revives', 'rideDistance', 'roadKills', 'swimDistance', 
                'teamKills', 'vehicleDestroys', 'walkDistance', 'weaponsAcquired', 'winPoints']]

In [ ]:
y = train_data['winPlacePerc']

In [ ]:
test_data = pd.read_csv('../input/test.csv')

In [ ]:
test_data.columns

In [ ]:
test_data.head()

In [ ]:
submissions = pd.read_csv('../input/sample_submission.csv')

In [ ]:
submissions.head()

In [ ]:
pred_feat = test_data[['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills',
                'killStreaks', 'longestKill', 'maxPlace', 'numGroups', 'revives', 'rideDistance', 'roadKills', 'swimDistance', 
                'teamKills', 'vehicleDestroys', 'walkDistance', 'weaponsAcquired', 'winPoints']]

## Distribution of Target Variable

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))
y.plot.hist(bins=50)
plt.show()

In [ ]:
print('Average:', y.mean())
print('Mode:', y.mode())

In [ ]:
def insight(x):
    print('Average: {:.4f}'.format(train_data[x].mean()))
    print('99% quantile: {} '.format(train_data[x].quantile(0.99)))
    print('Max: {}'.format(train_data[x].max()))

## Kills

In [ ]:
insight('kills')

**Distribution:**

In [ ]:
data = train_data['kills'].copy()
data[data > 7] = '8+'
plt.figure(figsize=(20,8))
sns.countplot(data.astype('str').sort_values())
plt.title("Kill Count",fontsize=15)
plt.show()

Most people don't kill even 1... How much do they knockout, damage or assist

In [ ]:
data = train_data[['kills', 'damageDealt', 'assists', 'DBNOs', 'winPlacePerc']].copy()
data = data[data['kills']==0]
plt.figure(figsize=(20,8))
plt.title("Knockouts by 0 killers",fontsize=15)
sns.distplot(data['DBNOs'])
plt.show()

plt.figure(figsize=(20,8))
plt.title("Assists by 0 killers",fontsize=15)
sns.distplot(data['assists'])
plt.show()

plt.figure(figsize=(20,8))
plt.title("Damage Dealt by 0 killers",fontsize=15)
sns.distplot(data['damageDealt'])
plt.show()

Most of these didn't even assist, knockout or deal some damage

In [ ]:
win_zerokills = len(data[data['winPlacePerc'] == 1])

In [ ]:
total_players = len(train_data)

In [ ]:
win_zerodamage = len(train_data[(train_data['damageDealt'] == 0) & (train_data['winPlacePerc'] == 1)])

In [ ]:
win_zeroassist = len(train_data[(train_data['assists'] == 0) & (train_data['winPlacePerc'] == 1)])

In [ ]:
win_zeroknocks = len(train_data[(train_data['DBNOs'] == 0) & (train_data['winPlacePerc'] == 1)])

In [ ]:
print('{} ({:.4f}%) players won without even one kill'.format(win_zerokills, win_zerokills*100/total_players))

In [ ]:
print('{} ({:.4f}%) players won without even knocking someone out'.format(win_zeroknocks, win_zeroknocks*100/total_players))

In [ ]:
print('{} ({:.4f}%) players won without even assisting'.format(win_zeroassist, win_zeroassist*100/total_players))

In [ ]:
print('{} ({:.4f}%) players won without dealing some damage'.format(win_zerodamage, win_zerodamage*100/total_players))

**Win percentage v/s kills**

In [ ]:
sns.jointplot(x='winPlacePerc', y='kills', data=train_data)
plt.show()
sns.jointplot(x='winPlacePerc', y='assists', data=train_data)
plt.show()
sns.jointplot(x='winPlacePerc', y='DBNOs', data=train_data)
plt.show()
sns.jointplot(x='winPlacePerc', y='damageDealt', data=train_data)
plt.show()

**Correlation between killing and winning**

In [ ]:
sns.heatmap(train_data[['kills', 'assists', 'DBNOs', 'damageDealt', 'winPlacePerc']].corr(), cmap='RdBu',annot=True)
plt.show()

In [ ]:
insight('DBNOs')

In [ ]:
insight('assists')

In [ ]:
insight('damageDealt')

## Boosts Heals Revives

** Insights **

In [ ]:
insight('boosts')

In [ ]:
insight('heals')

In [ ]:
insight('revives')

In [ ]:
data = train_data[['winPlacePerc', 'kills']].copy()
for x in ['boosts', 'heals', 'revives']:
    data1 = train_data[x].copy()
    print(data1.unique())
    data1[data1 > (data1.quantile(0.99)-1)] = (str(int(data1.quantile(0.99)))+'+')
    print(data1.unique())
    data = pd.concat([data, data1], axis=1)

In [ ]:
def heal_fix(x):
    if x != '11+' and x != 10:
        return '0'+str(x)
    if x == 10:
        return '10'
    else:
        return x
    

In [ ]:
data['heals'] = data['heals'].apply(heal_fix)

In [ ]:
data.head()

** Distribution **

In [ ]:
plt.figure(figsize=(20,8))
sns.countplot(data['boosts'].astype('str').sort_values())
plt.title("No. of Boosts Used",fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(20,8))
sns.countplot(data['heals'].astype('str').sort_values())
plt.title("No. of Heals",fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(20,8))
sns.countplot(data['revives'].astype('str').sort_values())
plt.title("No. of Revives",fontsize=15)
plt.show()

** Correlation **

In [ ]:
sns.heatmap(train_data[['boosts', 'heals', 'revives', 'winPlacePerc']].corr(), cmap='RdBu',annot=True)
plt.show()

In [ ]:
sns.jointplot(x='winPlacePerc', y='boosts', data=train_data)
plt.show()
sns.jointplot(x='winPlacePerc', y='heals', data=train_data)
plt.show()
sns.jointplot(x='winPlacePerc', y='revives', data=train_data)
plt.show()

##  Ride Swim Walk

** Insights **

In [ ]:
insight('walkDistance')

In [ ]:
insight('rideDistance')

In [ ]:
insight('swimDistance')

** Distribution **

In [ ]:
data = train_data['walkDistance'].copy()
data = data[data < data.quantile(0.99)]
plt.figure(figsize=(20,8))
sns.distplot(data)
plt.title("Walk Distance",fontsize=15)
plt.show()

In [ ]:
print('{} ({:.4f}%) walked 0 meters,which means they were killed before even taking a step'
      .format(len(train_data[train_data['walkDistance'] == 0]),
              len(train_data[train_data['walkDistance'] == 0])*100/len(train_data)))

In [ ]:
data = train_data['rideDistance'].copy()
data = data[data < data.quantile(0.99)]
plt.figure(figsize=(20,8))
sns.distplot(data)
plt.title("Ride Distance",fontsize=15)
plt.show()

In [ ]:
print("{} ({:.4f}%) rode 0 meters,which means they couldn't find a vehicle"
      .format(len(train_data[train_data['rideDistance'] == 0]),
              len(train_data[train_data['rideDistance'] == 0])*100/len(train_data)))

In [ ]:
data = train_data['swimDistance'].copy()
data = data[data < data.quantile(0.99)]
plt.figure(figsize=(20,8))
sns.distplot(data)
plt.title("Swim Distance",fontsize=15)
plt.show()

In [ ]:
print("{} ({:.4f}%) swim 0 meters,which means they need not to swim"
      .format(len(train_data[train_data['swimDistance'] == 0]),
              len(train_data[train_data['swimDistance'] == 0])*100/len(train_data)))

** Correlation **

In [ ]:
sns.heatmap(train_data[['walkDistance', 'rideDistance', 'swimDistance', 'winPlacePerc', 'kills']].corr(), cmap='RdBu',annot=True)
plt.show()

Walk Distance is highly correlated with winning

In [ ]:
sns.jointplot(x='winPlacePerc', y='walkDistance', data=train_data)
plt.show()
sns.jointplot(x='winPlacePerc', y='rideDistance', data=train_data)
plt.show()
sns.jointplot(x='winPlacePerc', y='swimDistance', data=train_data)
plt.show()

## Team Kills, Head shot Kills, Longest Kill, Road Kills, Vehicle Kills, Kill Streaks

** Insights **

In [ ]:
insight('headshotKills')

In [ ]:
insight('killStreaks')

In [ ]:
insight('longestKill')

In [ ]:
insight('roadKills')

In [ ]:
insight('teamKills')

In [ ]:
insight('vehicleDestroys')

** Distribution **

In [ ]:
data = train_data['headshotKills'].copy()
# data = data[data < data.quantile(0.99)]
plt.figure(figsize=(20,8))
sns.distplot(data)
plt.title("Head Shots",fontsize=15)
plt.show()

In [ ]:
print('{} ({:.4f}%) had 0 headshots'
      .format(len(train_data[train_data['headshotKills'] == 0]),
              len(train_data[train_data['headshotKills'] == 0])*100/len(train_data)))

In [ ]:
data = train_data['killStreaks'].copy()
# data = data[data < data.quantile(0.99)]
plt.figure(figsize=(20,8))
sns.distplot(data)
plt.title("Kill Streaks",fontsize=15)
plt.show()

In [ ]:
print('{} ({:.4f}%) had kill streaks less than 3'
      .format(len(train_data[train_data['killStreaks'] <= 3]),
              len(train_data[train_data['killStreaks'] <= 3])*100/len(train_data)))

In [ ]:
data = train_data['longestKill'].copy()
# data = data[data < data.quantile(0.99)]
plt.figure(figsize=(20,8))
sns.distplot(data)
plt.title("Longest Kill",fontsize=15)
plt.show()

In [ ]:
print('{} ({:.4f}%) had kill from a distance less than 50 meters'
      .format(len(train_data[train_data['longestKill'] <= 50]),
              len(train_data[train_data['longestKill'] <= 50])*100/len(train_data)))

In [ ]:
data = train_data['roadKills'].copy()
# data = data[data < data.quantile(0.99)]
plt.figure(figsize=(20,8))
sns.distplot(data)
plt.title("Road Kills",fontsize=15)
plt.show()

In [ ]:
print('{} ({:.4f}%) had 0 road kills'
      .format(len(train_data[train_data['roadKills'] == 0]),
              len(train_data[train_data['roadKills'] == 0])*100/len(train_data)))

In [ ]:
data = train_data['teamKills'].copy()
# data = data[data < data.quantile(0.99)]
plt.figure(figsize=(20,8))
sns.distplot(data)
plt.title("Teams Kills",fontsize=15)
plt.show()

In [ ]:
print('{} ({:.4f}%) had 0 team kills'
      .format(len(train_data[train_data['teamKills'] == 0]),
              len(train_data[train_data['teamKills'] == 0])*100/len(train_data)))

In [ ]:
data = train_data['vehicleDestroys'].copy()
# data = data[data < data.quantile(0.99)]
plt.figure(figsize=(20,8))
sns.distplot(data)
plt.title("Vehicles Destroyed",fontsize=15)
plt.show()

In [ ]:
print('{} ({:.4f}%) had destroyed 0 vehicles'
      .format(len(train_data[train_data['vehicleDestroys'] == 0]),
              len(train_data[train_data['vehicleDestroys'] == 0])*100/len(train_data)))

** Correlation **

In [ ]:
sns.heatmap(train_data[['headshotKills', 'killStreaks', 'longestKill', 'roadKills', 'teamKills', 'vehicleDestroys', 
                        'winPlacePerc', 'kills']].corr(), cmap='RdBu',annot=True)
plt.show()

In [ ]:
sns.jointplot(x='winPlacePerc', y='headshotKills', data=train_data)
plt.show()
sns.jointplot(x='winPlacePerc', y='killStreaks', data=train_data)
plt.show()
sns.jointplot(x='winPlacePerc', y='longestKill', data=train_data)
plt.show()
sns.jointplot(x='winPlacePerc', y='roadKills', data=train_data)
plt.show()
sns.jointplot(x='winPlacePerc', y='teamKills', data=train_data)
plt.show()
sns.jointplot(x='winPlacePerc', y='vehicleDestroys', data=train_data)
plt.show()

## Weapons Acquired, Kills, Kill place, Kill points

** Insights **

In [ ]:
insight('weaponsAcquired')

In [ ]:
insight('killPlace')

In [ ]:
insight('killPoints')

** Distribution **

In [ ]:
data = train_data['weaponsAcquired'].copy()
# data = data[data < data.quantile(0.99)]
plt.figure(figsize=(20,8))
sns.distplot(data)
plt.title("Weapons Acquired",fontsize=15)
plt.show()

In [ ]:
print("{} ({:.4f}%) couldn't even acquire a weapon"
      .format(len(train_data[train_data['weaponsAcquired'] == 0]),
              len(train_data[train_data['weaponsAcquired'] == 0])*100/len(train_data)))

In [ ]:
data = train_data['killPlace'].copy()
# data = data[data < data.quantile(0.99)]
plt.figure(figsize=(20,8))
sns.distplot(data)
plt.title("Kill Place",fontsize=15)
plt.show()

In [ ]:
data = train_data['killPoints'].copy()
# data = data[data < data.quantile(0.99)]
plt.figure(figsize=(20,8))
sns.distplot(data)
plt.title("Kill Points",fontsize=15)
plt.show()

In [ ]:
print("{} ({:.4f}%) had 1000 kill points"
      .format(len(train_data[train_data['killPoints'] == 1000]),
              len(train_data[train_data['killPoints'] == 1000])*100/len(train_data)))

** Correlation **

In [ ]:
sns.heatmap(train_data[['weaponsAcquired', 'killPlace', 'killPoints', 
                        'winPlacePerc', 'kills']].corr(), cmap='RdBu',annot=True)
plt.show()

In [ ]:
sns.jointplot(x='winPlacePerc', y='weaponsAcquired', data=train_data)
plt.show()
sns.jointplot(x='winPlacePerc', y='killPlace', data=train_data)
plt.show()
sns.jointplot(x='winPlacePerc', y='killPoints', data=train_data)
plt.show()

## Solos Duos Squads

In [ ]:
solos = train_data[train_data['numGroups']>50]
duos = train_data[(train_data['numGroups']>25) & (train_data['numGroups']<=50)]
squads = train_data[train_data['numGroups']<=25]
print("There are {} ({:.2f}%) solo games,\n\t  {} ({:.2f}%) duo games and\n\t  {} ({:.2f}%) squad games."
      .format(len(solos), 100*len(solos)/len(train_data),
              len(duos), 100*len(duos)/len(train_data),
              len(squads), 100*len(squads)/len(train_data),))

In [ ]:
fig,ax = plt.subplots(figsize =(20,10))
sns.pointplot(x='kills',y='winPlacePerc',data=solos,color='black',alpha=0.8)
sns.pointplot(x='kills',y='winPlacePerc',data=duos,color='#CC0000',alpha=0.8)
sns.pointplot(x='kills',y='winPlacePerc',data=squads,color='#3399FF',alpha=0.8)
plt.text(37,0.6,'Solos',color='black',fontsize = 17,style = 'italic')
plt.text(37,0.55,'Duos',color='#CC0000',fontsize = 17,style = 'italic')
plt.text(37,0.5,'Squads',color='#3399FF',fontsize = 17,style = 'italic')
plt.xlabel('Number of kills',fontsize = 15,color='blue')
plt.ylabel('Win Percentage',fontsize = 15,color='blue')
plt.title('Solo vs Duo vs Squad Kills',fontsize = 20,color='blue')
plt.grid()
plt.show()

Solo and Duo behave pretty mush the same

## Scatterplot of various features with target Variable

In [ ]:
for x in X.columns.drop(['kills', 'assists', 'DBNOs', 'damageDealt',
                         'boosts', 'heals', 'revives', 
                         'walkDistance', 'rideDistance', 'swimDistance',
                        'headshotKills', 'killStreaks', 'longestKill', 'roadKills', 'teamKills', 'vehicleDestroys',
                        'weaponsAcquired', 'killPlace', 'killPoints']):
    sns.jointplot(x='winPlacePerc',y=x, data=train_data )
    plt.show()

## Correlation

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
sns.heatmap(train_data.corr(), cmap='RdBu',annot=True)
plt.show()

In [ ]:
k = 5 #number of variables for heatmap
fig,ax = plt.subplots(figsize=(11, 11))
cols = train_data.corr().nlargest(k, 'winPlacePerc')['winPlacePerc'].index
cm = np.corrcoef(train_data[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10},
                 yticklabels=cols.values, xticklabels=cols.values)
plt.show()

# Preprocessing

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X))

In [ ]:
scaler_pred = MinMaxScaler()
pred_feat = pd.DataFrame(scaler_pred.fit_transform(pred_feat))

# Model Training

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
model = LinearRegression()
model.fit(X, y)
pred = model.predict(pred_feat)

submissions['winPlacePerc'] = pred
submissions.to_csv('LinearRegression.csv', index=False)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
model = RandomForestRegressor()
model.fit(X, y)
pred = model.predict(pred_feat)

submissions['winPlacePerc'] = pred
submissions.to_csv('RandomForestRegressor.csv', index=False)

## Keras Neural Network

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout

### [22->11->1]

In [ ]:
def NeuralNetwork1():
    model = Sequential()
    model.add(Dense(11, input_dim=22, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
network1 = NeuralNetwork1()

In [ ]:
network1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)

### [22->44->1]

In [ ]:
def NeuralNetwork2():
    model = Sequential()
    model.add(Dense(44, input_dim=22, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
network2 = NeuralNetwork2()

In [ ]:
network2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)

In [ ]:
network = NeuralNetwork2()
network.fit(X, y, epochs=10)
pred = network.predict(pred_feat)

In [ ]:
for p in pred[:5]:
    print(p)

In [ ]:
submissions['winPlacePerc'] = pred
submissions.to_csv('Keras22-44-1.csv', index=False)

### [22->44->22->11->1]

In [ ]:
def NeuralNetwork3():
    model = Sequential()
    model.add(Dense(44, input_dim=22, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(22, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(11, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
network3 = NeuralNetwork3()

In [ ]:
network3.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)

### [22->512->256->128->1]

In [ ]:
def NeuralNetwork4():
    model = Sequential()
    model.add(Dense(512, input_dim=22, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
network4 = NeuralNetwork4()

In [ ]:
network4.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)

In [ ]:
network = NeuralNetwork4()
network.fit(X, y, epochs=10)
pred = network.predict(pred_feat)

In [ ]:
submissions['winPlacePerc'] = pred
submissions.to_csv('Keras22-512-256-128-1.csv', index=False)